In [1]:
import glob
import os
import geopandas as gpd
import pandas as pd
import rasterio
import requests
import rioxarray as rio
from rasterio.enums import Resampling
from rasterio.features import shapes
from rasterio.merge import merge

# Get harvested areas from SR16beta

In [2]:
# Folder to save raw raster tiles
raw_path = r"/home/jovyan/shared/common/oslofjord_modelling/nibio_skog/data/sr16beta/raw"

# Merged tiff to create
tif_path = r"/home/jovyan/shared/common/oslofjord_modelling/nibio_skog/data/sr16beta/sr16beta_hogst_aar.tif"

# Vector data to create
vec_path = r"/home/jovyan/shared/common/oslofjord_modelling/nibio_skog/data/sr16beta/sr16beta.gpkg"
layer = "hogst_aar"

## 1. Download data

In [ ]:
# List of fylke numbers. From here:
# https://kart8.nibio.no/nedlasting/dashboard
areas = [42, 34, 15, 18, 3, 11, 54, 50, 38, 46, 30]
for area in areas:
    download_url = f"https://kart8.nibio.no/uttak_Download/sr16beta/{area:02d}_25832_SR16BETA-v21022023_RASTER.zip"
    response = requests.get(download_url, stream=True)
    
    if response.status_code != 200:
        raise Exception(
            f"Failed to download data for area {area}. Status code: {response.status_code}"
        )

    # If the download was successful, save the data to a local file
    zip_path = os.path.join(raw_path, f"{area}_25832_SR16BETA-v21022023_RASTER.zip")
    with open(zip_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

Finally, open a terminal in `raw_path` and run `unzip "./*.zip"` to unzip all the downloaded archives.

## 2. Merge

In [3]:
# Read separate tiles
search_path = os.path.join(raw_path, "sr16_*_SRRHOGSTAARbeta.tif")
flist = glob.glob(search_path)
src_files_to_mosaic = []
for fp in flist:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

mosaic, out_trans = merge(src_files_to_mosaic)

# Close files
for src in src_files_to_mosaic:
    src.close()

# Convert data type to int16
mosaic = mosaic.astype("int16")

# Keep only values after 2000
nodata_value = -9999
mosaic[~(mosaic > 2000)] = nodata_value

# Update metadata
out_meta = src.meta.copy()
out_meta.update(
    {
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "compress": "lzw",
        "BIGTIFF": "YES",
        "dtype": "int16",
        "nodata": nodata_value,
    }
)

# Write the mosaic raster to disk
with rasterio.open(tif_path, "w", **out_meta) as dest:
    dest.write(mosaic)

/tmp/ipykernel_1601/1351007077.py:14: RuntimeWarning: invalid value encountered in cast
  mosaic = mosaic.astype("int16")


## 3. Convert to vector

In [4]:
# Convert to vector
with rasterio.open(tif_path) as src:
    # Read mosaic
    crs = src.crs
    image = src.read(1)
    
    # Create shapes (polygons) from the raster
    results = (
        {"properties": {"year": int(v)}, "geometry": s}
        for s, v in shapes(image, transform=src.transform)
    )

# Convert the shapes to a GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(list(results), crs=crs)
gdf = gdf.query("year > 0")
gdf

,geometry,year
0,"POLYGON ((1190416.000 7967072.000, 1190432.000...",2018
1,"POLYGON ((1190224.000 7966832.000, 1190224.000...",2018
2,"POLYGON ((1190240.000 7954528.000, 1190240.000...",2018
3,"POLYGON ((1257680.000 7953760.000, 1257680.000...",2021
4,"POLYGON ((1257696.000 7953744.000, 1257696.000...",2021
...,...,...
423669,"POLYGON ((412528.000 6427888.000, 412528.000 6...",2020
423670,"POLYGON ((412512.000 6427856.000, 412512.000 6...",2020
423671,"POLYGON ((412016.000 6427840.000, 412016.000 6...",2021
423672,"POLYGON ((412448.000 6427776.000, 412480.000 6...",2018


In [5]:
# Save
gdf.to_file(vec_path, layer=layer, driver="GPKG")

In [6]:
# # Save
# gdf.to_file(
#     r"/home/jovyan/shared/common/oslofjord_modelling/nibio_skog/data/sr16beta/hogst_aar.shp"
# )